In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from pathlib import Path
import re
from readprocess import readprocess
from NNfunctionality import *
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [2]:
hist_path = Path(Path.cwd(),"..","ProcessedHistograms").resolve()
yield_path = Path(Path.cwd(),"Data","Yield Data","Crop_Yields.csv").resolve()
data_class = readprocess(featurepath=hist_path,yieldpath=yield_path)

In [3]:
dataset = prepare_dataset(data_class.final_dataset,data_class.yield_dict['Kenya'])
train_loader = DataLoader(dataset,batch_size=1)

In [4]:
epoch_no = 10
num_hidden_units = 100
num_features = 576 #64 bins * 9 bands per image in each time step
batch_size = 1
trainvalidate = TrainingValidatingLSTM()
model = LSTMnetwork(576,num_hidden_units)
mse_loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.001)
for i in range(epoch_no):
    trainvalidate.train(dataloader=train_loader,model=model,loss_fn=mse_loss,optimizer = optimizer,batch_size=batch_size)

C:\Users\Pasindu.Samaranayake\Desktop\MSc\MSBD 5001\EarthEngine\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.75 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [14]:
result = list()
ground = list()
h = model.init_hidden(1)
for batch,(X,y) in enumerate(train_loader):
    print(X.shape)
    break
    # res,_ = model(X,h)
    # result.append(float(res))
    # ground.append(float(y))

torch.Size([1, 29, 576])


In [12]:
ground

[5.440000057220459,
 8.34000015258789,
 6.630000114440918,
 8.180000305175781,
 7.760000228881836,
 6.360000133514404,
 7.329999923706055,
 5.579999923706055,
 6.139999866485596,
 8.010000228881836,
 7.059999942779541,
 6.929999828338623,
 6.360000133514404]

In [ ]:
lstm_layer = nn.LSTM(576,100,num_layers = 1,dropout = 0.75,batch_first = True)
dense_layer1 = nn.Linear(in_features=100,out_features=256,bias = True)
dense_layer2 = nn.Linear(in_features=256,out_features=1,bias=True)

In [ ]:
h = model.init_hidden(1)
for batch,(X,y) in enumerate(train_loader):
    l,h = lstm_layer(X,h)
    l = l.contiguous().view(-1,100)[-1]
    print(l)
    o1 = dense_layer1(l)
    print(o1)
    o2 = dense_layer2(o1)
    print(o2)
    # yield_val = model(X,h)
    #print(yield_val)
    break

tensor([ 0.1220, -0.0533, -0.0012, -0.0084,  0.0326, -0.0050,  0.0467, -0.0647,
         0.0825,  0.0339,  0.0589,  0.0801,  0.0063,  0.0733, -0.0119, -0.0060,
        -0.0248, -0.0939,  0.0455,  0.0948, -0.0487, -0.0288, -0.0707, -0.0468,
         0.0437,  0.0983, -0.1005, -0.0054, -0.0834, -0.0687, -0.0366, -0.0142,
         0.0127,  0.0331, -0.0728, -0.0497, -0.0896, -0.0240,  0.0031, -0.0088,
        -0.0376,  0.0331, -0.0342,  0.0043,  0.0258, -0.0658,  0.0717, -0.0260,
         0.0961,  0.0197, -0.0242,  0.0475,  0.0299,  0.0102,  0.0482, -0.0811,
         0.0866, -0.0542, -0.0331,  0.0113,  0.0539,  0.1102,  0.0736, -0.1080,
        -0.0440, -0.0526, -0.0030,  0.1028,  0.0584,  0.0621, -0.0550,  0.0381,
         0.0302,  0.0928, -0.1148,  0.0194, -0.0691, -0.0810, -0.0981, -0.1099,
         0.1043, -0.1049,  0.0082,  0.0879,  0.0600,  0.0133, -0.0823,  0.0305,
         0.0711,  0.0133, -0.0357,  0.0735, -0.0318, -0.0073, -0.0181,  0.0095,
        -0.0983, -0.0368, -0.0154,  0.03

In [ ]:
tuple(i.detach() for i in h)

(tensor([[[ 0.1220, -0.0533, -0.0012, -0.0084,  0.0326, -0.0050,  0.0467,
           -0.0647,  0.0825,  0.0339,  0.0589,  0.0801,  0.0063,  0.0733,
           -0.0119, -0.0060, -0.0248, -0.0939,  0.0455,  0.0948, -0.0487,
           -0.0288, -0.0707, -0.0468,  0.0437,  0.0983, -0.1005, -0.0054,
           -0.0834, -0.0687, -0.0366, -0.0142,  0.0127,  0.0331, -0.0728,
           -0.0497, -0.0896, -0.0240,  0.0031, -0.0088, -0.0376,  0.0331,
           -0.0342,  0.0043,  0.0258, -0.0658,  0.0717, -0.0260,  0.0961,
            0.0197, -0.0242,  0.0475,  0.0299,  0.0102,  0.0482, -0.0811,
            0.0866, -0.0542, -0.0331,  0.0113,  0.0539,  0.1102,  0.0736,
           -0.1080, -0.0440, -0.0526, -0.0030,  0.1028,  0.0584,  0.0621,
           -0.0550,  0.0381,  0.0302,  0.0928, -0.1148,  0.0194, -0.0691,
           -0.0810, -0.0981, -0.1099,  0.1043, -0.1049,  0.0082,  0.0879,
            0.0600,  0.0133, -0.0823,  0.0305,  0.0711,  0.0133, -0.0357,
            0.0735, -0.0318, -0.0073, 